In [95]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
# pip install git+https://github.com/alemartinello/dstapi
from dstapi import DstApi

import ipywidgets as widgets
from ipywidgets import TwoByTwoLayout
from IPython.display import display,clear_output
import matplotlib.pyplot as plt
import matplotlib
import time

import pandas as pd

In [99]:
import dataprojectv12 as dp

ModuleNotFoundError: No module named 'dataprojectv12'

In [98]:
REG = dp.import_REG()

AttributeError: module 'dataproject' has no attribute 'import_REG'

In [87]:
def import_REG():

    # connecting to dst
    ind = DstApi('REGK11')

    # choosing which variables to import
    params = {'table':'REGK11',
         'format':"BULK",
         'variables':[{'code':'OMRÅDE','values':['*']},
                     {'code':'PRISENHED','values':['INDL']},
                     {'code':'DRANST','values':['1']},
                     {'code':'FUNK1','values':['4']},
                     {'code':'ART','values':['TOT']},
                     {'code':'TID','values':['>2007']}]}
    # importing data
    data = ind.get_data(params)

    # dropping irrelevant and/or constant columns
    data.drop(columns=['PRISENHED', 'DRANST', 'FUNK1', 'ART'],inplace=True)

    # renaming columns
    data.rename(columns={'INDHOLD':'SUNDHEDSUDGIFTER'},inplace=True)

    return data

In [56]:
ind = DstApi('FOLK1A') # connecting to dst API
tabsum = ind.tablesummary(language='en')
tabsum  # presenting summary of data

Table FOLK1A: Population at the first day of the quarter by region, sex, age, marital status and time
Last update: 2023-02-10T08:00:00


,variable name,# values,First value,First value label,Last value,Last value label,Time variable
0,OMRÅDE,105,000,All Denmark,851,Aalborg,False
1,KØN,3,TOT,Total,2,Women,False
2,ALDER,127,IALT,Total,125,125 years,False
3,CIVILSTAND,5,TOT,Total,F,Divorced,False
4,Tid,61,2008K1,2008Q1,2023K1,2023Q1,True


In [58]:
# we are only interested in municipalities and not regions, so we loop thrugh all variables
# in 'OMRÅDER' and store the ID of municipalities
kom = []
for row, value in zip(ind.variable_levels('OMRÅDE')['id'],ind.variable_levels('OMRÅDE')['text']):
    if 'Region' not in value:
            kom.append(row)

Table FOLK1A: Population at the first day of the quarter by region, sex, age, marital status and time
Last update: 2023-02-10T08:00:00


In [59]:
# importing data
params = {'table':'FOLK1A',
         'format':"BULK",
         'variables':[{'code':'OMRÅDE','values':['*']},
                     {'code':'KØN','values':['*']},
                     {'code':'ALDER','values':['*']},
                     {'code':'Tid','values':['*']},
                     {'code':'CIVILSTAND','values':['TOT']}]}
data = ind.get_data(params)

In [61]:
def change_format(inn):
    return inn.replace('K','Q')
# data = pd.read_csv('FOLK1A',index_col=[0])
data['TID'] = pd.to_datetime(data.reset_index()['TID'].apply(change_format))
#data.resample('Y').last()
data = data[data['KØN']=='I alt'][['TID','OMRÅDE','ALDER','INDHOLD']]
data['ÅR'] = data['TID'].dt.year
data = data.groupby(['ÅR','OMRÅDE','ALDER']).last()
data = data.reset_index()
data = data.drop(columns = ['TID'])
data['ÅR'] = data['ÅR'].astype(int)
data = data[data['ALDER'] != 'I alt']
data['ALDER'] = data['ALDER'].str.strip(' år')
data['ALDER'] = data['ALDER'].astype(int)

In [62]:
REG = pd.read_csv('REGK11.csv',index_col=0)
BEF = data

In [63]:
BEF.rename(columns={'ÅR':'TID'},inplace=True)
new_df = pd.merge(BEF, REG,  how='left', left_on=['OMRÅDE','TID'], right_on = ['OMRÅDE','TID'])
samlet_bef = new_df.groupby(['TID','OMRÅDE'])['INDHOLD'].sum()
new_df['GAMLE'] = new_df['INDHOLD']
new_df.loc[new_df['ALDER']<60,'GAMLE'] = 0
samlet_bef1 = new_df.groupby(['TID','OMRÅDE'])['GAMLE'].sum()
df_ = pd.merge(samlet_bef, samlet_bef1,  how='left', left_on=['OMRÅDE','TID'], right_on = ['OMRÅDE','TID'])
df_['ANDEL'] = df_['GAMLE']/df_['INDHOLD']
df_.rename(columns={'INDHOLD':'SAMLET BEFOLKNING'},inplace=True)
df_ = pd.merge(df_,REG,how='left', left_on=['OMRÅDE','TID'], right_on = ['OMRÅDE','TID'])

In [64]:
df_

,OMRÅDE,TID,SAMLET BEFOLKNING,GAMLE,ANDEL,SUNDHEDSUDGIFTER
0,Aabenraa,2008,60284,14561,0.241540,2635.0
1,Aalborg,2008,195145,42502,0.217797,2393.0
2,Aarhus,2008,298711,53213,0.178142,2394.0
3,Albertslund,2008,27566,5414,0.196401,2598.0
4,Allerød,2008,23515,5253,0.223389,2618.0
...,...,...,...,...,...,...
1675,Vejle,2023,120949,30525,0.252379,NaN
1676,Vesthimmerlands,2023,36431,11754,0.322637,NaN
1677,Viborg,2023,97731,27318,0.279522,NaN
1678,Vordingborg,2023,45441,16655,0.366519,NaN


In [83]:
def plot_graph1(change=None):
    plot_graph_1(output_widget)

def plot_graph2(change=None):
    plot_graph_2(output_widget2)

def plot_graph_1(output):
    year = year_slider.value
    municipal = municipal_slider.value
    
    # Filter the DataFrame based on the selected year and municipal
    filtered_df = data[data['TID'] == year]
    filtered_df = filtered_df[filtered_df['OMRÅDE'].isin(municipal)]

    # Create the bar plot
    plt.barh(filtered_df['ALDER'].unique(), filtered_df['INDHOLD'])
    plt.title(f"Values for year {year}")
    plt.xlabel("Antal")
    plt.ylabel("År")
    plt.tight_layout()

    # Display the graph in the output widget
    with output:
        output.clear_output(wait=True)
        display(plt.gcf())
        
    plt.clf()
    
def plot_graph_2(output):

    year = year_slider.value
    municipal = municipal_slider.value
    
    filtered_df = df_[df_['OMRÅDE'].isin(municipal)]
    
    xx = filtered_df['ANDEL']
    yy = filtered_df['SUNDHEDSUDGIFTER']
    
    plt.scatter(xx,yy)

    for i, txt in enumerate(filtered_df['TID'].unique()):
        plt.annotate(txt, (xx.values[i], yy.values[i]))
    plt.xlabel('Share of population $\geq$60 years')
    plt.ylabel('Health care expenditure per capita, DKK')
    ax = plt.gca()
    ax.get_yaxis().set_major_formatter(matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))
        
    with output:
        output.clear_output(wait=True)
        display(plt.gcf())

        
    plt.clf()

# Create the year slider
year_slider = widgets.IntSlider(
    value=data['TID'].min(),
    min=data['TID'].min(),
    max=data['TID'].max(),
    step=1,
    description="Year:",
    continuous_update=True,
)

# Create the municipal slider
municipal_slider = widgets.SelectMultiple(
    options=data['OMRÅDE'].unique(),
    value=['Ærø'],
    description='Municipal:',
    disabled=False,
)

# Create the output widgets
output_widget = widgets.Output()
output_widget2 = widgets.Output()

# Observe the slider changes and update the graphs
year_slider.observe(plot_graph1, names='value')
municipal_slider.observe(plot_graph1, names='value')

year_slider.observe(plot_graph2, names='value')
municipal_slider.observe(plot_graph2, names='value')

# Create a 2x2 GridBox layout
layout = widgets.GridBox(
    children=[year_slider, municipal_slider, output_widget, output_widget2],
    layout=widgets.Layout(
        width='100%',
        grid_template_columns='50% 50%',
        grid_template_rows='25% 50%',
        grid_gap='10px',
        grid_template_areas='''
            "year_slider municipal_slider"
            "output_widget output_widget2"
        ''')
)

# Initialize the graphs
plot_graph1()
plot_graph2()

# Display the GridBox layout
display(layout)


GridBox(children=(IntSlider(value=2008, description='Year:', max=2023, min=2008), SelectMultiple(description='…

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>